In [110]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

True


In [111]:
with open("cmn.txt", encoding='utf8') as f:
    lines = f.readlines()

sep = ".!?,。？！，"
lines = ["".join([char if char not in sep else " " + char + " " for char in line]) for line in lines]
print(lines[0])
lines = [line.split("\t")[:2] for line in lines]
lines = [(line_pair[0].split(), line_pair[1].split()) for line_pair in lines]

source_sequence = []
target_sequence = []
for line in lines:
    source_sequence.append(line[0])
    target = []
    for seq in line[1]:
        if len(seq) == 1:
            target.append(seq)
        else:
            target.extend([char for char in seq])
    target_sequence.append(target)
print(source_sequence[10], target_sequence[10])

source_sequence = [["<bos>"] + sequence + ["<eos>"] for sequence in source_sequence]
target_sequence = [["<bos>"] + sequence + ["<eos>"] for sequence in target_sequence]

def padding_sequence(padding_size, sequence):
    if len(sequence) > padding_size:
        return sequence[:padding_size]
    elif len(sequence) < padding_size:
        for i in range(len(sequence), padding_size):
            sequence.append("<pad>")
    return sequence
    
source_sequence = [padding_sequence(20, sequence) for sequence in source_sequence]
target_sequence = [padding_sequence(20, sequence) for sequence in target_sequence]

source_count_dict = {}
target_count_dict = {}

for sequence in source_sequence:
    for word in sequence:
        if source_count_dict.get(word, -1) == -1:
            source_count_dict[word] = 0
        else:
            source_count_dict[word] += 1
for sequence in target_sequence:
    for word in sequence:
        if target_count_dict.get(word, -1) == -1:
            target_count_dict[word] = 0
        else:
            target_count_dict[word] += 1

target_words = ["<unknown>", "<pad>", "<eos>", "<bos>"]
source_words = ["<unknown>", "<pad>", "<eos>", "<bos>"]
target_dict = {"<unknown>":0, "<pad>":1, "<eos>":2, "<bos>":3}
source_dict = {"<unknown>":0, "<pad>":1, "<eos>":2, "<bos>":3}
print(target_dict["<bos>"], target_dict["<eos>"])
for k in source_count_dict:
    if source_count_dict[k] < 2 or source_dict.get(k, -1) != -1:
        continue
    source_dict[k] = len(source_words)
    source_words.append(k)
for k in target_count_dict:
    if target_count_dict[k] < 2 or target_dict.get(k, -1) != -1:
        continue
    target_dict[k] = len(target_words)
    target_words.append(k)

print(target_dict["<bos>"], target_dict["<eos>"])
print(len(source_dict), len(target_dict))

Hi . 	嗨 。 	CC-BY 2 . 0 (France) Attribution: tatoeba . org #538123 (CM) & #891077 (Martha)

['Oh', 'no', '!'] ['不', '会', '吧', '。']
3 2
3 2
3629 2635


In [116]:
class SeqDataset(Dataset):
    def __init__(self, source_lines, target_lines, source_words, target_words, source_dict, target_dict) -> None:
        super().__init__()
        self.source_lines = source_lines
        self.target_lines = target_lines
        self.source_words = source_words
        self.target_words = target_words
        self.source_dict = source_dict
        self.target_dict = target_dict
    def __getitem__(self, index):
        line = self.source_lines[index]
        source_index = [self.source_dict.get(word, 0) for word in line]
        line = self.target_lines[index]
        target_index = [self.target_dict.get(word, 0)  for word in line]
        # here we use tensor, so we can get [batch_size, sequence_length] tensor. or we will get wierd things.
        return torch.tensor(source_index).cuda(), torch.tensor(target_index).cuda()
    def __len__(self):
        return len(self.source_lines)
seq_data_set = SeqDataset(source_lines=source_sequence,
                          target_lines=target_sequence,
                          source_words=source_words,
                          target_words=target_words,
                          source_dict=source_dict,
                          target_dict=target_dict)
print(seq_data_set[0])
train_data_loader = DataLoader(dataset=seq_data_set, batch_size=2, shuffle=True)
for iter in train_data_loader:
    # print(iter[0].shape, iter[1].shape)
    print(iter)
    break


(tensor([3, 4, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0'), tensor([3, 4, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0'))
[tensor([[   3,  864,  955,  226, 2421,  466, 1961,    5,    2,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   3,   11,  245,    0,    0,    0,  501,  375, 2066,    5,    2,    1,
            1,    1,    1,    1,    1,    1,    1,    1]], device='cuda:0'), tensor([[   3,   36,   10,  834,  168,  163, 2066,  245,  446,    5,    2,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   3,   19,  348,  122,    0,    0,  696,    0,    5,    2,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]], device='cuda:0')]


In [117]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        raise NotImplementedError

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
    def init_state(self, state):
        raise NotImplementedError
    def forward(self, x, state):
        raise NotImplementedError

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder_ = encoder
        self.decoder_ = decoder
    def forward(self, x, y):
        encoder_output = self.encoder_(x)
        state = self.decoder_.init_state(encoder_output)
        decoder_output = self.decoder_(y, state)
        return decoder_output
    # def predict(self, x):
    #     NotImplementedError

In [120]:
class GRUEncoder(Encoder):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.hidden_dim = embedding_dim * 2
        self.num_layers = 2
        self.word2vec = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=self.hidden_dim, num_layers=self.num_layers, bias=True, dropout=0.5)
    def forward(self, x):
        hidden_state = torch.zeros((self.num_layers, x.shape[1], self.hidden_dim)).cuda()
        embedding = self.word2vec(x)
        # print("embedding x ", embedding.shape, hidden_state.shape)
        output, hidden = self.gru(embedding, hidden_state)
        return hidden

class GRUDecoder(Decoder):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.hidden_dim = embedding_dim * 2 
        self.num_layers = 2
        self.word2vec = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=self.hidden_dim, num_layers=self.num_layers, bias=True, dropout=0.5)
        self.linear = nn.Linear(self.hidden_dim, vocab_size)    
    def init_state(self, state):
        return state
    def forward(self, y, state):
        embedding = self.word2vec(y)
        output, hidden = self.gru(embedding, state)
        # print("embedding y ", embedding.shape)
        # print("output y ", output.shape)
        # output.shape = sequence, batch, embedding_dim
        one_hot = self.linear(output)
        return torch.sigmoid(one_hot)

In [122]:
def train(data_loader, seq2seq, epoch, cross_loss):
    optimizer = torch.optim.Adam(seq2seq.parameters(), lr=0.001)
    for _ in range(epoch):
        for data in data_loader:
            x, y_label = data
            # x, y shape=[batch, sequence length], here we permute to make their shape=[sequence length, batch]
            x = x.permute(1, 0)
            y_label = y_label.permute(1, 0)
            # y_inference shape=[sequence length, batch, vocab_size]
            # print("x, y shape: ", x.shape, y_label.shape)
            y_inference = seq2seq(x, y_label)
            # print("y_inference ", y_inference.shape)
            # change shape for cross entropy
            y_label = y_label.permute(1, 0)
            y_inference = y_inference.permute(1, 2, 0)
            # print("Y label: ", y_label[0], "y_inference: ", y_inference[0][:][:])
            loss = cross_loss(y_inference, y_label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print("loss ", loss.shape)

seq_data_set = SeqDataset(source_lines=source_sequence,
                          target_lines=target_sequence,
                          source_words=source_words,
                          target_words=target_words,
                          source_dict=source_dict,
                          target_dict=target_dict)
train_data_loader = DataLoader(dataset=seq_data_set, batch_size=32, shuffle=True)
seq2seq = Seq2Seq(GRUEncoder(len(source_words), 96), GRUDecoder(len(target_words), 96))
seq2seq.to(device)
cross_loss = nn.CrossEntropyLoss()
train(train_data_loader, seq2seq, 1, cross_loss)

loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.Size([])
loss  torch.S

KeyboardInterrupt: 

In [40]:
seq_data_set = SeqDataset(source_lines=source_sequence,
                          target_lines=target_sequence,
                          source_words=source_words,
                          target_words=target_words,
                          source_dict=source_dict,
                          target_dict=target_dict)
train_data_loader = DataLoader(dataset=seq_data_set, batch_size=32, shuffle=True)

seq2seq = Seq2Seq(GRUEncoder(len(source_words), 96), GRUDecoder(len(target_words), 96))

In [65]:
embedding = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
class EmbeddingDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
    def __getitem__(self, index):
        return embedding[index]
    def __len__(self):
        return len(embedding)

data_loader = DataLoader(dataset=EmbeddingDataset(), batch_size=2)
for data in data_loader:
    print(data)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[7, 8, 9]])


In [123]:
import math
-math.log(math.exp(0) / (math.exp(1) * 2 + math.exp(0)))

1.861994804058251

In [136]:
def sequence_mask(X, valid_len, value=0):
    """在序列中屏蔽不相关的项"""
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype=torch.float32,
                        device=X.device)[None, :] < valid_len[:, None]
    X[~mask] = value
    return X
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    """带遮蔽的softmax交叉熵损失函数"""
    # pred的形状：(batch_size,num_steps,vocab_size)
    # label的形状：(batch_size,num_steps)
    # valid_len的形状：(batch_size,)
    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)
        self.reduction='none'
        unweighted_loss = super(MaskedSoftmaxCELoss, self).forward(
            pred.permute(0, 2, 1), label)
        print("unweighted", unweighted_loss)
        weighted_loss = (unweighted_loss * weights).mean(dim=1)
        return weighted_loss
loss = MaskedSoftmaxCELoss()
print(loss(torch.ones(3, 4, 10), torch.ones((3, 4), dtype=torch.long),
     torch.tensor([4, 2, 0])))
print(nn.CrossEntropyLoss(reduction="none")(torch.ones(3, 10, 4), torch.ones((3, 4), dtype=torch.long)))

unweighted tensor([[2.3026, 2.3026, 2.3026, 2.3026],
        [2.3026, 2.3026, 2.3026, 2.3026],
        [2.3026, 2.3026, 2.3026, 2.3026]])
tensor([2.3026, 1.1513, 0.0000])
tensor([[2.3026, 2.3026, 2.3026, 2.3026],
        [2.3026, 2.3026, 2.3026, 2.3026],
        [2.3026, 2.3026, 2.3026, 2.3026]])
